In [ ]:
!pip3 install tabula-py numpy pandas pyPDF2 requests beautifulsoup4 tqdm

In [ ]:
import tabula
import numpy as np
import pandas as pd
import os 
from pathlib import Path
import PyPDF2
import re
import requests
import json
import time
import requests
from bs4 import BeautifulSoup as bs4
import urllib
from multiprocessing import Pool,Manager,Queue
from tqdm import tqdm

In [ ]:
req=requests.get("http://ggsipu.ac.in/ExamResults/ExamResultsmain.htm")

bs=bs4(req.text)

ahrefs=[i.attrs['href'] for i in bs.table.find_all("a")]

In [ ]:
try:
    os.mkdir("files")
except FileExistsError:
    pass
os.chdir("files")

In [ ]:
for i in range(len(ahrefs)):
    ahrefs[i]='http://ggsipu.ac.in/ExamResults/'+urllib.parse.unquote(ahrefs[i])
ahrefs

In [ ]:
filenames=os.listdir()

In [ ]:
scheme_reg = re.compile(r'scheme\s+of\s+examinations',re.IGNORECASE)
institution_reg = re.compile(r'institution\s*:\s*([\w\n(,)\-.& ]+)S\.No',re.IGNORECASE)
sem_reg = re.compile(r'se\s?m[.//\w\n]+:\s+([\w\n]+)',re.IGNORECASE)
programme_reg = re.compile(r'programme\s+name:\s+([\w(,)\-& \n]+)SchemeID',re.IGNORECASE)
branch_reg = re.compile(r'[\w ,\-&]+\(([\w, \-]+)\)')

In [ ]:
def filter_text(text):
    return ' '.join(text.strip().split()).replace('&','and').replace('.','').title()
filter_text('Dr. Akhilesh Das Gupta Institute Of Technology And Management(Formerly Niec)')

In [ ]:
def get_info(text) :
    college = filter_text(institution_reg.search(text)[1])
    semester = int(sem_reg.search(text)[1].strip())
    course = filter_text(programme_reg.search(text)[1])
    try:
        branch = filter_text(branch_reg.search(course)[1])
    except TypeError:
        branch = "None"
    if('(' in course):
        course = course[0:course.find('(')].strip()
    info = {
        'college' : college,
        'semester' : semester,
        'course' : course,
        'branch' : branch,
    }
    return info

In [ ]:
## Debug:
text="Paper Code (Credit***)InternalTotal (Grade**)LEGENDExternal** If Grade Based*** If Credit BasedResult Prepration Date : 16-JUN-20SCHEME OF EXAMINATIONSPrg. Code: 888      Programme: BACHELOR OF COMMERCE (HONOURS)      SchemeID: 148882017001      Sem./Annual: FIRST SEMESTERS. No.PaperIDPaper CodePaper NameCreditInstitution Code: 123      Institution: DELHI INSTITUTE OF ADVANCED STUDIESTypeKindExamModeMinorMajorMax. MarksPass Marks1888101BCOM101MANAGEMENT PROCESS AND ORGANIZATIONAL BEHAVIOUR4THEORYUESCOMPULSORYDROPPABLE2575100402888103BCOM103FINANCIAL ACCOUNTING4THEORYUESCOMPULSORYDROPPABLE2575100403888105BCOM105MICRO ECONOMICS4THEORYUESCOMPULSORYDROPPABLE2575100404888107BCOM107BUSINESS COMMUNICATION4THEORYUESCOMPULSORYDROPPABLE2575100405888109BCOM109COMPUTER APPLICATIONS4THEORYUESCOMPULSORYDROPPABLE2575100406888111BCOM111COMPUTER APPLICATIONS LAB2PRACTICALUESCOMPULSORYDROPPABLE406010040Printed On: 19-Jun-2020Page No.: 1"
print(text)
get_info(text)

In [ ]:
SITE = "https://api-rhapsody.herokuapp.com/academia"
# SITE = "http://localhost:3000/academia"

In [ ]:
# #Add college
# data ={ 
#     'college' : {
#         'college' : "University School Of Information, Communication & Technology (Formerly Usit)"
#         }
# }
# r = requests.post(SITE+"/college",json=data)
# print(r,r.content)


In [ ]:
def already_exists(info) :
    r = requests.get(SITE+"/semester",params=info)
    content = json.loads(r.content)
#     print(r.status_code,r.content)
    return r.status_code == 200 and content != {}

def already_exists_college(info):
    r = requests.get(SITE+"/college",params=info)
    content = json.loads(r.content)
#     print(r.status_code,r.content)
    return r.status_code == 200 and content != {}

    

In [ ]:
def getSubjects(df) :
    subjects = []
    for index,row in df.iterrows() :
        subject = {}
        subject['subject'] = row['Subject'].strip().title()
        subject['subjectCode'] = row['Code']
        subject['credits'] = row['Credit']
        subjects.append(subject)
    return subjects

In [ ]:
def pdfextract(filename,queue,colleges,infos):
    sno=queue.get()
    success=True
    try:
      pdf = PyPDF2.PdfFileReader(filename)
    except Exception as e:
      print(f"{sno}:{filename} PDF error")
      return False
    pages=pdf.getNumPages()
    for i in range(0,pages) :
        text = pdf.getPage(i).extractText()
        text=text.replace("\n",'')
        if scheme_reg.search(text) :
            try:
                info = get_info(text)
                if  info in infos or already_exists(info):
                    print(f"{sno}:already exists {info}")
                    if info not in infos:
                        infos.append(info)
                    continue
                df = tabula.read_pdf(filename,pages=i+1)
                subjects = getSubjects(df[0])     
            except Exception as e:
                success=False
                print(f"{sno}:{filename}:{i}")
                print(f"{text}")
                traceback.print_exc() 
                break
            #     print(e)
            #     continue
            print(f"{sno}:adding {info}")
            info['semester'] = {'semester' : info['semester'], 'subjects' : subjects}
            data ={ 
                'college' : {
                    'college' : info["college"]
                    }
            }
            if(info['college'] not in colleges and not already_exists_college(info)):
                print(f"{sno}:adding college {info['college']}")
                r = requests.post(SITE+"/college",json=data)
                colleges.append(info['college'])
            r = requests.post(SITE+"/semester",json=info)
            if r.status_code!=200:
                print(f"{sno}:{filename}:{i} status_code:{r.status_code}\n\t{r}:{r.content}\n\t{info}")
                if('Semester already exist' in r.content.decode()):
                    infos.append(info)
                    if(info['college'] not in colleges):
                        colleges.append(info['college'])
            else:
                print(f"{sno}: added {info} success")
    return success

In [ ]:
def download(_):
    url,filename,queue,colleges,infos= _
    filename=str(filename)+".pdf"
    r=requests.get(url,allow_redirects=True)
    f=open(filename,'wb')
    f.write(r.content)
    f.close()
    _=pdfextract(filename,queue,colleges,infos)
    if(_==True):
        os.remove(filename)

In [ ]:
from itertools import repeat
import traceback
import pandas as pd
start=277
end=278
with Manager() as manager:
    queue = manager.Queue()
    colleges = manager.list()
    infos = manager.list()
    for i in range(start,end,1):
        queue.put(i)
    with Pool(8) as pool:
        for _ in pool.imap_unordered(download,list(zip(ahrefs[start:end],range(start,end,1),repeat(queue,end-start),repeat(colleges,end-start),repeat(infos,end-start)))):
          pass